In [1]:
import datetime
import time
import math
import pymysql
from sqlalchemy import create_engine 
import numpy as np
import pandas as pd
from importlib import reload 


import tushare as ts
ts.set_token('11137efdbeac800606d677871a55b3fd5aef79890c59867a8f34d03e')
pro = ts.pro_api()

''' 自建库 '''
from qflib import basic
reload(basic)

global engine, conn
engine = basic.engine()
conn = basic.conn(basic.engine())

In [5]:
df = pd.DataFrame({'sz_max_10': [-5, 3, 12, -1, 8, 0]})

bins = [-float('inf'), 0, 5, 10, float('inf')] 
labels = ['<0', '0-5', '0-10', '>10']
df['sz_max_10_bin'] = pd.cut(df['sz_max_10'], bins=bins, labels=labels)
df['sz_max_10_bin'].value_counts()

<0      3
0-5     1
0-10    1
>10     1
Name: sz_max_10_bin, dtype: int64

In [3]:
df

,sz_max_10,sz_max_10_bin
0,-5,<0
1,3,0-10
2,12,>10
3,-1,<0
4,8,0-10
5,0,<0


In [19]:
# df.info()
# del df['display_name']
pi2degree = 180/math.pi
print( math.tan(45), math.tan(45)*pi2degree , math.tan(0.1), math.tan(0.2))

1.6197751905438615 92.806282178162 0.10033467208545055 0.2027100355086725


In [3]:
a = np.array([0,30,45,60,90])
print ('不同角度的正弦值：')
# 通过乘 pi/180 转化为弧度  
print (np.sin(a*np.pi/180))
print ('\n')
print ('数组中角度的余弦值：')
print (np.cos(a*np.pi/180))
print ('\n')
print ('数组中角度的正切值：')
print (np.tan(a*np.pi/180))



不同角度的正弦值：
[0.         0.5        0.70710678 0.8660254  1.        ]


数组中角度的余弦值：
[1.00000000e+00 8.66025404e-01 7.07106781e-01 5.00000000e-01
 6.12323400e-17]


数组中角度的正切值：
[0.00000000e+00 5.77350269e-01 1.00000000e+00 1.73205081e+00
 1.63312394e+16]


In [ ]:
a = np.array([0,30,45,60,90])  
print ('含有正弦值的数组：')
sin = np.sin(a*np.pi/180)  
print (sin)
print ('\n')
print ('计算角度的反正弦，返回值以弧度为单位：')
inv = np.arcsin(sin)  
print (inv)
print ('\n')
print ('通过转化为角度制来检查结果：')
print (np.degrees(inv))
print ('\n')
print ('arccos 和 arctan 函数行为类似：')
cos = np.cos(a*np.pi/180)  
print (cos)
print ('\n')
print ('反余弦：')
inv = np.arccos(cos)  
print (inv)
print ('\n')
print ('角度制单位：')
print (np.degrees(inv))
print ('\n')


In [ ]:
a = np.array([0,30,45,60,90])  
print ('tan 函数：')
tan = np.tan(a*np.pi/180)  
print (tan)
print ('\n')
print ('反正切：')
inv = np.arctan(tan)  
print (inv)
print ('\n')
print ('角度制单位：')
print (np.degrees(inv))


In [10]:

np.degrees(np.arctan(1)), np.degrees(np.arctan(0.001321)), np.degrees(np.arctan(0.001449))

(45.0, 0.07568768071076688, 0.08302152641046219)

In [ ]:
# 从 tran_daily 中读出5条
sql="SELECT * FROM tran_daily where trade_date='20230710' LIMIT 1"
df = pd.read_sql_query(sql, conn)
df

In [ ]:
# 写入到 tran_000, append 模式
# write_no = df.to_sql('tran_000', conn, index=False, if_exists='append', chunksize=2) # 20000个，每天成交几千只股票，足够了
write_no = df.to_sql('day_zhibiao', conn, index=False, if_exists='append', chunksize=2000) # 20000个，每天成交几千只股票，足够了
write_no


In [ ]:
# df.insert(loc=2, column='new_col', value=3) 
# del df['index']
# df.loc[[0], 'ts_code'] = 'testcode'

In [ ]:
# 从 tran_000 读出数据
sql2 = 'select * from tran_000'
df2 = pd.read_sql_query(sql2, conn, index_col=None)
df2

In [ ]:
# 验证 exec_sql
sql1 = 'create table test_000( ts_code varchar(12), trade_date varchar(10))'
sql2 = 'drop table if exists test'
status = basic.exec_sql(conn,sql2)
status


In [ ]:
# 从 tran_000 读出数据
sql3 = "select * from tran_daily where trade_date='20230717' "
df3 = pd.read_sql_query(sql3, conn, index_col=None)
df3.shape

to_sql(self, name, con, schema=None, if_exists='fail', index=True,
       index_label=None, chunksize=None, dtype=None, method=None):
- if_exists: 
    - append : append到表中，不存在表时，就新建表；
    - replace : 无论是否存在，直接新建表
- index：对DataFrame的index索引的处理，为True时索引也将作为数据写入数据表
- index_label:当上一个参数index为True时，设置写入数据表时index的列名称
- chunsize：设置整数，如20000，一次写入数据时的数据行数量，当数据量很大时，需要设置，否则会链接超时写入失败。
    - 每日交易不超过7000支股票，20000万足够了。
- 

index=True,
           index_label=None, chunksize=None, dtype=None, method=None):



In [ ]:
import pandas as pd


l1 =["1", "1", "1", "2", "2", "2", "2", "2"]
l2 =[1, 2, 2, 2, 2, 2, 2, 3]
l3 =[45, 25, 28, 70, 95, 98, 120, 80]
cowmast = pd.DataFrame(list(zip(l1, l2, l3))) 
cowmast.columns =['Cow', 'Lact', 'DIM']

In [ ]:
cowmast 


In [ ]:

def rolling_count(val):
    if val <= 2:
        rolling_count.count += 1
    else:
        # rolling_count.previous = val
        rolling_count.count = 0
    return rolling_count.count
# def rolling_count(val):
#     if val == rolling_count.previous:
#         rolling_count.count +=1
#     else:
#         rolling_count.previous = val
#         rolling_count.count = 1
#     return rolling_count.count

rolling_count.count = 0 #static variable
cowmast['under60'] = cowmast['new'].apply(rolling_count) #new column in dataframe
cowmast

In [ ]:
rolling_count.count

In [ ]:

# cowmast['xmast'] = cowmast['Cow'].apply(rolling_count) #new column in dataframe



In [ ]:

cowmast['t2'] = cowmast[['DIM', 'Lact']].apply( lambda x: x["DIM"] - x["Lact"], axis=1 ) 
# df["E"]     = df[["A","C"]].apply(lambda x: x["A"] + x["C"], axis=1)


In [ ]:
cowmast['new']=cowmast['t2'].rolling(3).count( cowmast['t2']>0  )


In [ ]:

cowmast



In [ ]:
cowmast['t1'] = cowmast['DIM'] + cowmast['Lact'] 

In [ ]:
cowmast